In [1]:
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

from collection import s3_download_file
from preprocessing import delete_corrupted, preprocess
from prediction import predict

torch.manual_seed(0)

In [ ]:
s3_download_file("kagglecatsanddogs_5340.zip", ".cache/data.zip")

In [ ]:
!unzip -q -n data.zip

In [2]:
# Directory path of your dataset
data_dir = 'PetImages'

delete_corrupted(data_dir)

Deleted 0 corrupted images.


In [3]:
train_loader, val_loader, dataset = preprocess(data_dir)

# Verify the class labels of the dataset
print(dataset.classes)

# Verify sizes
for images, labels in train_loader:
    print("Size of first batch")
    print("Features size:", images.size())
    print("Labels size:", labels.size())
    break

['Cat', 'Dog']
torch.Size([32, 3, 50, 50]) torch.Size([32])


In each iteration, the data loader returns a tuple of two batches (images and labels).
The images batch contains 32 images.
The labels batch contains the corresponding 32 labels for those images.

Each image has 50x50 pixels, with 3 color channels (RGB).

In [5]:
# Define the model (use a pre-trained ResNet and modify the final layer)
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
# Modify final layer to define 2 output classes: Cat and Dog
model.fc = nn.Linear(num_features, 2)

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("Using {device} for training")

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Epoch 1/10, 
 Training Loss: 0.4139
Validation Loss: 0.4070, Accuracy: 0.7826
Epoch 2/10, 
 Training Loss: 0.2825
Validation Loss: 0.3051, Accuracy: 0.8706
Epoch 3/10, 
 Training Loss: 0.2249
Validation Loss: 0.3125, Accuracy: 0.8704
Epoch 4/10, 
 Training Loss: 0.1852
Validation Loss: 0.3332, Accuracy: 0.8610
Epoch 5/10, 
 Training Loss: 0.1535
Validation Loss: 0.3493, Accuracy: 0.8748
Epoch 6/10, 
 Training Loss: 0.1294
Validation Loss: 0.2810, Accuracy: 0.8881
Epoch 7/10, 
 Training Loss: 0.0934
Validation Loss: 0.3358, Accuracy: 0.8913
Epoch 8/10, 
 Training Loss: 0.0891
Validation Loss: 0.4179, Accuracy: 0.8648
Epoch 9/10, 
 Training Loss: 0.0676
Validation Loss: 0.3852, Accuracy: 0.8746
Epoch 10/10, 
 Training Loss: 0.0659
Validation Loss: 0.3709, Accuracy: 0.8830


In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, \n Training Loss: {epoch_loss:.4f}')

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct.double() / len(val_loader.dataset)
    print(f' Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')

In [19]:
# Test with new image
from PIL import Image

result = predict("PetImages/Cat/0.jpg")
print(result)

AttributeError: 'Image' object has no attribute 'shape'

In [30]:
#first_batch_example = next(iter(val_loader))[0].to(device)
first_batch_example = torch.randn(1, 3, 50, 50).to(device)
version = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
torch.onnx.export(model, first_batch_example, f"cats-vs-dogs/v{version}/model.onnx")